In [63]:
# Have to install janitor package to run. Not included in anaconda.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor

In [64]:
exec(open("../header.py").read())

# Purpose

To replicate and then test out Boone and White (2015).

# Import raw data

In [65]:
sample = False
header = Header(sample)

In [66]:
raw_crsp = pd.read_csv(header.raw_root("crsp_data.csv"), parse_dates = ['date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3,7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [67]:
print(f"Date range: {min(raw_crsp.date)} - {max(raw_crsp.date)}")

Date range: 1994-01-03 00:00:00 - 2007-12-31 00:00:00


In [68]:
raw_crsp.head(5)

,PERMNO,date,SICCD,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,BID,ASK,SHROUT,NUMTRD,ewretd
0,10001,1994-01-03,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,17.750,4791.0,-0.034014,17.75,18.0,1091.0,4.0,0.003510
1,10001,1994-01-04,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,17.500,2000.0,-0.014085,17.25,18.0,1091.0,2.0,0.006372
2,10001,1994-01-05,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,18.000,800.0,0.028571,17.25,18.0,1091.0,2.0,0.005785
3,10001,1994-01-06,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,17.250,675.0,-0.041667,17.25,18.0,1091.0,1.0,0.002806
4,10001,1994-01-07,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,17.875,4200.0,0.036232,17.25,18.0,1091.0,2.0,0.005166


# Clean data

In [69]:
cleaned_crsp = raw_crsp\
    .clean_names()\
    .loc[lambda x:(x.date >= '1996-01-01')&(x.date <= '2006-12-31')]\
    .assign(mktcap = lambda x: x.prc * x.shrout)\
    .assign(year = lambda x:x.date.dt.year,
            month = lambda x:x.date.dt.month,
            yrmo = lambda x: x.year*100 + x.month)

In [70]:
cleaned_crsp['ranks'] = cleaned_crsp\
    .groupby('date', as_index = False)['mktcap']\
    .transform(lambda x:x.rank(ascending = False))

In [71]:
cleaned_crsp.head(5)

,permno,date,siccd,ncusip,ticker,comnam,permco,cusip,prc,vol,...,bid,ask,shrout,numtrd,ewretd,mktcap,year,month,yrmo,ranks
504,10001,1996-01-02,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,-9.125,0.0,...,8.75,9.5,2281.0,0.0,0.010979,-20814.125,1996,1,199601,8182.0
505,10001,1996-01-03,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,9.500,119.0,...,8.75,9.5,2281.0,1.0,0.005154,21669.500,1996,1,199601,6429.0
506,10001,1996-01-04,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,-9.125,0.0,...,8.75,9.5,2281.0,0.0,-0.005707,-20814.125,1996,1,199601,8143.0
507,10001,1996-01-05,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,8.750,1384.0,...,8.75,9.5,2281.0,2.0,0.005605,19958.750,1996,1,199601,6550.0
508,10001,1996-01-08,4920,29274A10,EWST,ENERGY WEST INC,7953,36720410,8.750,1000.0,...,8.75,9.5,2281.0,2.0,0.003672,19958.750,1996,1,199601,6051.0


# Data Integrity checks

In [74]:
print(f"Old shape: {raw_crsp.shape}")
print(f"New shape: {cleaned_crsp.shape}")

Old shape: (27927544, 16)
New shape: (21984131, 21)


In [80]:
test = cleaned_crsp\
    .groupby(['permno', 'date'], as_index = False)\
    .agg({'ticker':'count'})

assert max(test.ticker) == 1

In [82]:
test = cleaned_crsp\
    .groupby(['ticker', 'date'], as_index = False)\
    .agg(counts = ('permno','count'))

In [83]:
test

,ticker,date,counts
0,A,1996-05-23,1
1,A,1996-05-24,1
2,A,1996-05-28,1
3,A,1996-05-29,1
4,A,1996-05-30,1
...,...,...,...
21416118,ZZ,2006-12-22,1
21416119,ZZ,2006-12-26,1
21416120,ZZ,2006-12-27,1
21416121,ZZ,2006-12-28,1


# Save cleaned datsets

In [73]:
cleaned_crsp.to_csv(header.clean_root("crsp.csv"), index = False)